<a href="https://colab.research.google.com/github/alessandrow19/desafio-dio1/blob/main/dio_desafio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def fetch_and_clean_text(url)->str:
  try:
      response = requests.get(url)
      response.raise_for_status()
      soup = BeautifulSoup(response.content, 'html.parser')
      text = soup.get_text()

      cleaned_text = re.sub(r'\s+', ' ', text).strip()


      article_container = soup.find('div', class_='c-article-base') or soup.find('article')
      if article_container:
          article_text = article_container.get_text()

          cleaned_article_text = re.sub(r'\s+', ' ', article_text).strip()
          return cleaned_article_text
      else:

          return cleaned_text


  except requests.exceptions.RequestException as e:
      print(f"Error fetching the URL: {e}")
  except Exception as e:
      print(f"An error occurred: {e}")




In [ ]:
import os
from openai import AzureOpenAI
from google.colab import userdata

endpoint = "https://dio-azure-desafio.openai.azure.com/"
model_name = "o4-mini"
deployment = "o4-mini"

# Get the subscription key from Colab's Secrets Manager
subscription_key = userdata.get('AZURE_OPENAI_SUBSCRIPTION_KEY')
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)


def generate_response(text, lang):

   messages=[
        {
            "role": "system",
            "content": "Você é um especilista em traduzir textos ",
        },
        {
            "role": "user",
            "content": f"traduza o {text} para {lang} e responda em formato markdown",
        }
    ]


   response = client.chat.completions.create(
          messages=messages,
          max_completion_tokens=10000,
          model=deployment
   )

   print(response.choices[0].message.content)

In [ ]:
url=userdata.get('URL')


texto=fetch_and_clean_text(url)
generate_response(texto, "pt-br")

```markdown
# Ben Halpern para a equipe DEV  
**Publicado em 3 de outubro**

# Pré-carregando o feed e a barra lateral da página inicial do DEV (e do Forem) para benefícios substanciais de performance  
*#webperf #performance #frontend #webdev*

Quando você visita a página inicial do dev.to, ela é armazenada em cache na edge para que seja entregue o mais rápido possível pelo servidor mais próximo. Isso é ótimo, mas essa entrega estática também impede que mostremos imediatamente seu feed e sua barra lateral personalizados, baseados nas tags e pessoas que você segue. Para obter essas informações, buscamos seu feed de forma assíncrona, e essa requisição vai até o banco de dados para carregar a resposta dinamicamente. Esse é um padrão comum, mas é irritante ficar olhando para uma tela de carregamento — e a espera muitas vezes era maior do que gostaríamos.

Essa demora é causada por uma cascata de outros recursos que precisam ser carregados primeiro, como CSS, JavaScript e imagens.

## Uma 